Lets import dependencies

In [ ]:
! pip3 install ib_insync
! pip3 install pandas
! pip3 install openpyxl
! pip3 install matplotlib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ib_insync import *

from pandas import DataFrame

Setup IB

In [3]:
util.startLoop() # necessary for notebook

ib = IB()
ib.connect()
ib.reqMarketDataType(3)

Create the contracts

In [4]:
from ib_insync import contract

contractInfos = [
    {
        "conId": 296574745,
        "exchange": "NYMEX",
    },
    {
        "conId": 297249704,
        "exchange": "IPE",
    },
    {
        "conId": 7089,
        "exchange": "NYSE",
    },
    {
        "conId": 470458975,
        "exchange": "NYSE",
    },
    {
        "conId": 4215217,
        "exchange": "NYSE",
    },
    {
        "conId": 13977,
        "exchange": "NYSE",
    },
    {
        "conId": 6890,
        "exchange": "NYSE",
    },
    {
        "conId": 5684,
        "exchange": "NYSE",
    },
    {
        "conId": 754442,
        "exchange": "NYSE",
    },
    {
        "conId": 10885,
        "exchange": "NYSE",
    },
    {
        "conId": 57698865,
        "exchange": "NYSE",
    },
    {
        "conId": 39118796,
        "exchange": "NYSE",
    },
    {
        "conId": 9831,
        "exchange": "NYSE",
    },
    {
        "conId": 3142097,
        "exchange": "NYSE",
    },
    {
        "conId": 6608450,
        "exchange": "NYSE",
    },
    {
        "conId": 13805,
        "exchange": "NYSE",
    },
    {
        "conId": 75960201,
        "exchange": "NYSE",
    },
    {
        "conId": 418893644,
        "exchange": "NYSE",
    },
    {
        "conId": 10190340,
        "exchange": "NYSE",
    },
    {
        "conId": 415578515,
        "exchange": "NYSE",
    },
    {
        "conId": 495512572,
        "exchange": "CME",
    },
    {
        "conId": 551601503,
        "exchange": "CBOT",
    },
]

contracts = [contract.Contract(**info) for info in contractInfos]

for contract in contracts:
    ib.qualifyContracts(contract)

contracts[0]

Contract(secType='FUT', conId=296574745, symbol='CL', lastTradeDateOrContractMonth='20230120', multiplier='1000', exchange='NYMEX', currency='USD', localSymbol='CLG3', tradingClass='CL')

# EITHER
request all the initial data

In [38]:
import datetime
import os

query_date = datetime.datetime.now().strftime("%Y%m%d-00:00:00")

def request_data(contract):
    trades = util.df(ib.reqHistoricalData(contract, endDateTime=query_date, durationStr='30 D', barSizeSetting='10 mins', whatToShow='TRADES', useRTH=True))
    bid = util.df(ib.reqHistoricalData(contract, endDateTime=query_date, durationStr='30 D', barSizeSetting='10 mins', whatToShow='BID', useRTH=True))
    ask = util.df(ib.reqHistoricalData(contract, endDateTime=query_date, durationStr='30 D', barSizeSetting='10 mins', whatToShow='ASK', useRTH=True))
    avg = util.df(ib.reqHistoricalData(contract, endDateTime=query_date, durationStr='30 D', barSizeSetting='10 mins', whatToShow='BID_ASK', useRTH=True))
    volume = trades["volume"]
    last_trade = trades["close"]
    bid = bid["close"]
    ask = ask["close"]
    avg_bid = avg["open"]
    avg_ask = avg["close"]
    last_trade.name = "last_trade"
    bid.name = "bid"
    ask.name = "ask"
    avg_bid.name = "avg_bid"
    avg_ask.name = "avg_ask"
    bars = pd.concat([volume, bid, ask, avg_bid, avg_ask, last_trade], axis=1)
    return bars


bars = {contract.symbol: request_data(contract) for contract in contracts}


if not os.path.exists("data"):
    os.mkdir("data")

for symbol, bar in bars.items():
    bar.to_csv(f"data/{symbol}.csv")

print(bars["CL"].head())


   volume    bid    ask  avg_bid  avg_ask  last_trade
0   558.0  75.13  75.15   75.016   75.038       75.13
1   343.0  74.64  74.66   74.996   75.018       74.64
2   334.0  74.75  74.77   74.638   74.660       74.76
3   310.0  74.57  74.60   74.708   74.730       74.58
4   496.0  74.67  74.70   74.708   74.730       74.68


# OR
Just import the data

In [46]:
bars2 = {contract.symbol: pd.read_csv(f"data/{symbol}.csv", index_col=0, dtype="str") for contract in contracts}
bars2["CL"].head()

,volume,bid,ask,avg_bid,avg_ask,last_trade
0,119.000,33893.000,33901.000,33877.300,33877.300,33898.000
1,67.000,33801.000,33808.000,33868.700,33868.700,33806.000
2,109.000,33821.000,33828.000,33804.300,33804.300,33821.000
3,136.000,33893.000,33901.000,33868.600,33868.600,33889.000
4,486.000,33912.000,33915.000,33895.000,33895.000,33914.000


In [55]:
def add_triggers(bars):
    # Trigger 1: Volume
    bars["avg_volume"] = bars["volume"].mean()
    bars["volume_trigger"] = (bars["volume"] - bars["avg_volume"]) / bars["avg_volume"] * 100
    bars["volume_trigger"] = bars["volume_trigger"].apply(lambda x: 1 if x > 50 else -1 if x < -50 else 0)

    # Trigger 2: Spread
    bars["spread"] = bars["ask"] - bars["bid"]
    bars["avg_spread"] = bars["spread"].mean() # i
    bars["max_spread"] = bars["spread"].max() # ii
    bars["delta_spread"] = (bars["max_spread"] - bars["avg_spread"]) / 2 # iii
    bars["min_tolerance_band"] = (bars["avg_spread"] - bars["delta_spread"]).abs() # iv
    bars["max_tolerance_band"] = bars["avg_spread"] + bars["delta_spread"] # v
    buy = bars["spread"] < bars["min_tolerance_band"]
    sell = bars["spread"] > bars["max_tolerance_band"]
    bars["spread_trigger"] = 0
    bars.loc[buy, "spread_trigger"] = 1
    bars.loc[sell, "spread_trigger"] = -1

    # Trigger 3: Bid-Ask
    buy = bars["last_trade"] > bars["avg_ask"]
    sell = bars["last_trade"] < bars["avg_bid"]
    bars["bid_ask_trigger"] = 0
    bars.loc[buy, "bid_ask_trigger"] = 1
    bars.loc[sell, "bid_ask_trigger"] = -1

    # Trigger Sum
    bars["trigger_sum"] = bars["volume_trigger"] + bars["spread_trigger"] + bars["bid_ask_trigger"]
    bars["order"] = bars["trigger_sum"].apply(lambda x: "BUY" if x > 2 else "SELL" if x < -2 else "WAIT")
    return bars

triggers = {symbol: add_triggers(bar) for symbol, bar in bars.items()}

In [56]:
if not os.path.exists("outputs"):
    os.mkdir("outputs")

for symbol, trigger in triggers.items():
    trigger.to_csv(f"outputs/{symbol}.csv")